In [114]:
import csv, os, sys, json, datetime, collections
import numpy as np
import pandas as pd

In [4]:
sys.path.append('../vessel-scoring/vessel_scoring/')
sys.path.append('../vessel-scoring/')

In [18]:
import vessel_scoring.models
from vessel_scoring.models import train_model_on_data, load_data
from vessel_scoring import data, utils
import vessel_scoring.colspec
import vessel_scoring

In [6]:
import vessel_scoring.models
models = vessel_scoring.models.load_models()
#is_fishing = models['Logistic'].predict_proba(track_points)[:,1]

In [7]:
models.keys()

['Logistic opt MSE',
 'Logistic',
 'Legacy (12 Hour)',
 'Logistic--Longliner',
 'Logistic--Purse seine',
 'Legacy',
 'Logistic--Trawler']

## Transform track data to display

In [175]:
def track_format(track_data):
    result = []
    past_timestamp = 0
    for record in track_data:
        mmsi = str(int(record[0]))
        _type = "vessel"
        timestamp = int(record[1])
        if past_timestamp == timestamp:
            continue
            
        distance_from_shore = record[2]
        distance_from_port = record[3]
        speed = record[4]
        course = record[5]
        past_timestamp 
        latitude = record[6]
        longitude = record[7]
        tag = "iuu_" + str(mmsi)[0:8]
        this_json = {"uuid": mmsi,
                    "type": _type,
                    "latitude": latitude,
                    "longitude": longitude,
                    "timestamp": timestamp,
                    "tag": tag}
        past_timestamp = timestamp
        result.append(this_json)
        
    return result
    
track_format(this_track['x'][1:10])

(  8.48258570e+13,   1.32539264e+09,  0.,  6708.0390625,  0.,  5.30000019, -34.64751434, -58.34066772)
(  8.48258570e+13,   1.32539266e+09,  0.,  6708.0390625,  0.,  335., -34.64751053, -58.3406601)
(  8.48258570e+13,   1.32539838e+09,  0.,  6708.0390625,  0.,  47.79999924, -34.64749146, -58.34064102)
(  8.48258570e+13,   1.32540381e+09,  0.,  6708.0390625,  0.,  243., -34.64751816, -58.34064102)
(  8.48258570e+13,   1.32540382e+09,  0.,  6708.0390625,  0.,  244.8999939, -34.64752197, -58.3406601)
(  8.48258570e+13,   1.32540399e+09,  0.,  6708.0390625,  0.,  241.80000305, -34.64751434, -58.34062576)
(  8.48258570e+13,   1.32540427e+09,  0.,  6708.0390625,  0.,  246., -34.64752579, -58.34064484)
(  8.48258570e+13,   1.32540432e+09,  0.,  6708.0390625,  0.,  231.8999939, -34.6475029, -58.34066772)
(  8.48258570e+13,   1.32541021e+09,  0.,  6708.0390625,  0.,  185.3999939, -34.64749527, -58.34066772)


[{'latitude': -34.647514343300003,
  'longitude': -58.340667724600003,
  'tag': 'iuu_84825856',
  'timestamp': 1325392644,
  'type': 'vessel',
  'uuid': '84825856950400'},
 {'latitude': -34.647510528600002,
  'longitude': -58.340660095200001,
  'tag': 'iuu_84825856',
  'timestamp': 1325392665,
  'type': 'vessel',
  'uuid': '84825856950400'},
 {'latitude': -34.647491455100003,
  'longitude': -58.340641021700002,
  'tag': 'iuu_84825856',
  'timestamp': 1325398384,
  'type': 'vessel',
  'uuid': '84825856950400'},
 {'latitude': -34.647518157999997,
  'longitude': -58.340641021700002,
  'tag': 'iuu_84825856',
  'timestamp': 1325403813,
  'type': 'vessel',
  'uuid': '84825856950400'},
 {'latitude': -34.647521972699998,
  'longitude': -58.340660095200001,
  'tag': 'iuu_84825856',
  'timestamp': 1325403825,
  'type': 'vessel',
  'uuid': '84825856950400'},
 {'latitude': -34.647514343300003,
  'longitude': -58.340625762899997,
  'tag': 'iuu_84825856',
  'timestamp': 1325403993,
  'type': 'vessel

In [133]:
def unix_time(timestamp):
    value = datetime.datetime.fromtimestamp(timestamp)
    return value.strftime('%Y-%m-%d %H:%M:%S')

In [172]:
for file in os.listdir("/home/data/wayne_data/training-data/data/tracks"):
    if file.endswith(".npz"):
        this_track = np.load(os.path.join("/home/data/wayne_data/training-data/data/tracks", file))
        result = track_format(this_track['x'])
        start_time = 99999999999
        end_time = 0
        
        with open('track_to_show/' + file[:-4] + '.csv', 'w+') as f:
            fieldnames = result[0].keys()
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for record in result:
                if start_time > record['timestamp']:
                    start_time = record['timestamp']
                if end_time < record['timestamp']:
                    end_time = record['timestamp']
                writer.writerow(record)
        print unix_time(start_time),unix_time(end_time), len(result)

2013-05-30 07:35:49 2016-09-01 19:00:36 15199
2012-01-05 19:47:58 2016-08-26 01:19:40 107254
2012-01-01 08:26:39 2016-11-25 07:59:52 37742
2012-01-01 08:12:00 2015-04-14 05:56:08 83696
2012-03-04 15:53:42 2016-11-22 05:09:57 34185


KeyboardInterrupt: 

In [181]:
this_track['x'].dtype

dtype([('mmsi', '<f8'), ('timestamp', '<f8'), ('distance_from_shore', '<f8'), ('distance_from_port', '<f8'), ('speed', '<f8'), ('course', '<f8'), ('lat', '<f8'), ('lon', '<f8')])

In [100]:
#for index in range(1,200):
#    value = datetime.datetime.fromtimestamp(track_sample['x'][index][1])
#    print(value.strftime('%Y-%m-%d %H:%M:%S'))

## Read Labeled Data

In [281]:
labeled_np = np.load('/home/data/wayne_data/training-data/data/labeled/kristina_ps_Purse_seines.measures.labels.npz')
labeled_np['x'].shape

(146196,)

In [282]:
!mkdir -p datasets

In [283]:
labeled_df = pd.DataFrame(labeled_np['x'])

In [284]:
fieldnames = ['course',
 'distance_from_port',
 'distance_from_shore',
 'lat',
 'lon',
 'measure_cos_course',
 'measure_count_10800',
 'measure_count_1800',
 'measure_count_21600',
 'measure_count_3600',
 'measure_count_43200',
 'measure_count_86400',
 'measure_course',
 'measure_courseavg_10800',
 'measure_courseavg_1800',
 'measure_courseavg_21600',
 'measure_courseavg_3600',
 'measure_courseavg_43200',
 'measure_courseavg_86400',
 'measure_coursestddev_10800',
 'measure_coursestddev_10800_log',
 'measure_coursestddev_1800',
 'measure_coursestddev_1800_log',
 'measure_coursestddev_21600',
 'measure_coursestddev_21600_log',
 'measure_coursestddev_3600',
 'measure_coursestddev_3600_log',
 'measure_coursestddev_43200',
 'measure_coursestddev_43200_log',
 'measure_coursestddev_86400',
 'measure_coursestddev_86400_log',
 'measure_daylight',
 'measure_daylightavg_10800',
 'measure_daylightavg_1800',
 'measure_daylightavg_21600',
 'measure_daylightavg_3600',
 'measure_daylightavg_43200',
 'measure_daylightavg_86400',
 'measure_distance_from_port',
 'measure_latavg_10800',
 'measure_latavg_1800',
 'measure_latavg_21600',
 'measure_latavg_3600',
 'measure_latavg_43200',
 'measure_latavg_86400',
 'measure_lonavg_10800',
 'measure_lonavg_1800',
 'measure_lonavg_21600',
 'measure_lonavg_3600',
 'measure_lonavg_43200',
 'measure_lonavg_86400',
 'measure_pos_10800',
 'measure_pos_1800',
 'measure_pos_21600',
 'measure_pos_3600',
 'measure_pos_43200',
 'measure_pos_86400',
 'measure_sin_course',
 'measure_speed',
 'measure_speedavg_10800',
 'measure_speedavg_1800',
 'measure_speedavg_21600',
 'measure_speedavg_3600',
 'measure_speedavg_43200',
 'measure_speedavg_86400',
 'measure_speedstddev_10800',
 'measure_speedstddev_10800_log',
 'measure_speedstddev_1800',
 'measure_speedstddev_1800_log',
 'measure_speedstddev_21600',
 'measure_speedstddev_21600_log',
 'measure_speedstddev_3600',
 'measure_speedstddev_3600_log',
 'measure_speedstddev_43200',
 'measure_speedstddev_43200_log',
 'measure_speedstddev_86400',
 'measure_speedstddev_86400_log',
 'mmsi',
 'speed',
 'timestamp',
 'y']

In [285]:
len(fieldnames)

81

In [286]:
#fieldnames = list(labeled_df.columns)
with open('./datasets/labeled.csv', 'w+') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    for index, row in labeled_df.iterrows():
        this_dict = row.to_dict()
        y = this_dict.pop('is_fishing')
        this_dict['y'] = y
        writer.writerow(collections.OrderedDict(sorted(this_dict.items())))

## Start to train

In [287]:
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.externals import joblib

In [288]:
labeled_df.shape[0]

146196

In [289]:
X = np.zeros((labeled_df.shape[0], len(labeled_df.columns) - 1))
y = np.zeros((labeled_df.shape[0]))

In [290]:
for idx, row in labeled_df.iterrows():
    if row.isnull().values.any():
        continue
    this_dict = row.to_dict()
    is_fish = this_dict.pop('is_fishing')
    X[idx,] = [this_dict[elemt] for elemt in this_dict]
    y[idx] = is_fish

In [291]:
'''
with open('./datasets/labeled.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for idx, row in enumerate(reader):
        record = row
        X[idx,] = [float(elemt) for elemt in row[:-1]]
        y[idx] = int(float(row[-1]))
'''

"\nwith open('./datasets/labeled.csv', 'r') as f:\n    reader = csv.reader(f, delimiter=',')\n    for idx, row in enumerate(reader):\n        record = row\n        X[idx,] = [float(elemt) for elemt in row[:-1]]\n        y[idx] = int(float(row[-1]))\n"

In [292]:
model = LogisticRegression(class_weight=None)

In [293]:
model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [294]:
print(model.classes_)

[ 0.  1.]


In [295]:
pred = model.predict(X)

score = metrics.accuracy_score(y, pred)
print("accuracy:   %0.3f" % score)

print("confusion matrix:")
print(metrics.confusion_matrix(y, pred))

accuracy:   0.960
confusion matrix:
[[140316      0]
 [  5880      0]]


## Try XGboost

In [296]:
import xgboost as xgb

In [297]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X, y)
predictions = gbm.predict(X)

In [299]:
score = metrics.accuracy_score(y, predictions)
print("accuracy:   %0.3f" % score)

print("confusion matrix:")
print(metrics.confusion_matrix(y, predictions))

accuracy:   0.995
confusion matrix:
[[140126    190]
 [   481   5399]]
